## 기본 합성곱 신경망 구현

In [1]:
import tensorflow as tf
import numpy as np

## 하이퍼 파라미터

In [2]:
EPOCHS = 10

## 모델 정의

In [3]:
class ConvNet(tf.keras.Model):
    def __init__(self):
        super(ConvNet, self).__init__()
        conv2d = tf.keras.layers.Conv2D
        maxpool = tf.keras.layers.MaxPool2D
        self.sequence = list()
        # Conv2D(출력채널, 커널, 크기 유지, activation)
        self.sequence.append(conv2d(16, (3, 3), padding='same', activation='relu')) # 28x28x16
        self.sequence.append(conv2d(16, (3, 3), padding='same', activation='relu')) # 28x28x16
        self.sequence.append(maxpool((2, 2))) # 14x14x16
        self.sequence.append(conv2d(32, (3, 3), padding='same', activation='relu')) # 14x14x32
        self.sequence.append(conv2d(32, (3, 3), padding='same', activation='relu')) # 14x14x32
        self.sequence.append(maxpool((2, 2))) # 7x7x32
        self.sequence.append(conv2d(64, (3, 3), padding='same', activation='relu')) # 7x7x64
        self.sequence.append(conv2d(64, (3, 3), padding='same', activation='relu')) # 7x7x64
        self.sequence.append(tf.keras.layers.Flatten()) 
        self.sequence.append(tf.keras.layers.Dense(2048, activation='relu'))
        self.sequence.append(tf.keras.layers.Dense(10, activation='softmax'))
        
    def call(self, x, training=False, mask=None):
        for layer in self.sequence:
            x = layer(x)
        return x

## 학습, 테스트 루프 정의

In [4]:
# Implement training loop
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images)

    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

## 데이터셋 준비


In [5]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0 # float64

# x_train : (NUM_SAMPLE, 28, 28) -> (NUM_SAMPLE, 28, 28, 1) conv layer에서 작동을 위해
x_train = x_train[..., tf.newaxis].astype(np.float32) # float32로 바꿔야 tf에서 에러X
x_test = x_test[..., tf.newaxis].astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## 학습 환경 정의
### 모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

In [6]:
# Create model
model = ConvNet()

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## 학습 루프 동작

In [7]:
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)

    for test_images, test_labels in test_ds:
        test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.11396678537130356, Accuracy: 96.34000396728516, Test Loss: 0.05605003610253334, Test Accuracy: 98.18000030517578
Epoch 2, Loss: 0.04373960942029953, Accuracy: 98.66166687011719, Test Loss: 0.028335824608802795, Test Accuracy: 99.11000061035156
Epoch 3, Loss: 0.03361247479915619, Accuracy: 98.98333740234375, Test Loss: 0.03890087455511093, Test Accuracy: 98.97000122070312
Epoch 4, Loss: 0.026593226939439774, Accuracy: 99.17166900634766, Test Loss: 0.04007416218519211, Test Accuracy: 98.87999725341797
Epoch 5, Loss: 0.0227156151086092, Accuracy: 99.28666687011719, Test Loss: 0.03304038941860199, Test Accuracy: 99.15999603271484
Epoch 6, Loss: 0.021726438775658607, Accuracy: 99.35333251953125, Test Loss: 0.025926919654011726, Test Accuracy: 99.23999786376953
Epoch 7, Loss: 0.0172214787453413, Accuracy: 99.44833374023438, Test Loss: 0.032797448337078094, Test Accuracy: 99.16999816894531
Epoch 8, Loss: 0.016514211893081665, Accuracy: 99.48500061035156, Test Loss: 0.03909517